In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
news_train = pd.read_pickle('/content/drive/MyDrive/BTP_Minor/large/news_train_finbert.pkl')
# news_val = pd.read_pickle('/content/drive/MyDrive/BTP_Minor/large/news_val_finbert.pkl')
# news_test = pd.read_pickle('/content/drive/MyDrive/BTP_Minor/large/news_test_finbert.pkl')

In [ ]:
price_train = pd.read_pickle('/content/drive/MyDrive/BTP_Major/large/patchTST_finetune/patchTST_train.pkl')
# price_val = pd.read_pickle('/content/drive/MyDrive/BTP_Major/large/patchTST_finetune/patchTST_val.pkl')
# price_test = pd.read_pickle('/content/drive/MyDrive/BTP_Major/large/patchTST_finetune/patchTST_test.pkl')

In [ ]:
def compute(news, price):
  news['Date'] = news['Date'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d').strftime('%Y-%m-%d'))
  news = news.reset_index(drop=True)
  news.set_index('Date', inplace=True)

In [ ]:
compute(news_train, price_train)

In [ ]:
symbols = pd.read_csv('/content/drive/MyDrive/BTP_Minor/large/sp500.csv')
symbols = symbols.drop(['Unnamed: 0','Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded'], axis=1)
symbols

,Symbol
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
498,YUM
499,ZBRA
500,ZBH
501,ZION


In [ ]:
import pickle
path = '/content/drive/MyDrive/BTP_Minor/small/'
with open(path+'blank_emb.pkl', 'rb') as handle:
    emb_blanks = pickle.load(handle)

In [ ]:
array = emb_blanks['BERT'][0]
len(array)

768

In [ ]:
def merge(price_train, news_train):
  not_included = ['CE','KVUE','VLTO','CEG','GEHC','PEAK','WBD']

  for symbol in symbols['Symbol']:
      if symbol not in not_included:
          for index, value in price_train['{}'.format(symbol)].items():
              print("Embeddings for {} at {}".format(symbol, str(index)))
              print("Price Emb of "+ str(len(value)))

              if index in news_train.index:
                  print("News emb")
                  news_emb = news_train[symbol].loc[index]
                  print(len(news_emb))
              else:
                  print("Blank Emb")
                  news_emb = emb_blanks['BERT'][0]
                  print(len(news_emb))

              value = np.concatenate([value, news_emb]) if index in news_train.index else np.concatenate([value, emb_blanks['BERT'][0]])
              price_train.at[index, symbol] = value

In [ ]:
len(price_train['MMM'][0])

1408

In [ ]:
price_train.shape

(85, 496)

In [ ]:
merge(price_train, news_train)

Streaming output truncated to the last 5000 lines.
768
Embeddings for DVN at 2023-01-18
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-19
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-20
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-23
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-24
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-25
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-26
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-27
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-30
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-01-31
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-02-01
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-02-02
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-02-03
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-02-06
Price Emb of 640
Blank Emb
768
Embeddings for DVN at 2023-02-07
Price Emb of 640

In [ ]:
price_test.to_pickle('/content/drive/MyDrive/BTP_Major/large/merged_emb_news_patchTST_test.pkl')

In [ ]:
val = pd.read_pickle('/content/drive/MyDrive/BTP_Major/large/merged_emb_news_patchTST_val.pkl')

In [ ]:
val.shape

(43, 496)

In [ ]:
# Convert tensors in DataFrame columns to a list
X_train = price_test
tensor_list = [tf.concat([X_train[col][i] for col in X_train.columns], axis=0) for i in range(len(X_train))]

# Stack tensors from the list horizontally to create a single tensor
concatenated_tensor = tf.stack(tensor_list, axis=0)

x_train_tensor = tf.reshape(concatenated_tensor, (concatenated_tensor.shape[0], len(X_train.columns), -1))
x_train_tensor.shape

TensorShape([85, 496, 1408])